<a href="https://colab.research.google.com/github/norflin321/ml/blob/main/nano_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
steps = 3000
eval_interval = 300
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
eval_batches_num = 200
#----------------

torch.manual_seed(1337)
torch.device(device)

with open("datasets/tiny_shakespeare.txt", "r", encoding="utf-8") as file:
    text = file.read()

# here are all the unique characters that occur in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])
print(f"-- dataset has {vocab_size} unique characters:", itos)

# train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


# this function averages up the loss over multiple batches,
# for each train and val sets, we generate many batches of data (how many specified by eval_batches_num variable),
# then measure a loss for each of them and get their average.
# so this will be a lot less noisy, then a loss inside training loop for a random batch
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_batches_num)
        for k in range(eval_batches_num):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# BigramLanguageModel is a very simple model because the tokens are not talking to each other so given the previous context of
# what was generated we are only looking at the very last character to make a predictions about what comes next.
# To make better predictions for what comes next, tokens (characters) have to start talking to each other
# and figuring out what is in the context.
class BigramLanguageModel(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # create embeding table which is a tensor of shape [64, 64] filled with random numbers
        self.token_embedding_table = torch.nn.Embedding(vocab_size, vocab_size)
    def forward(self, idx, targets=None):
        # we cant just train a model on indices of characters from lookup table, before we need to embed them.
        # inputs shape is [batch_size, block_size] after embeding we should get a tensor
        # with shape [batch_size, block_size, 64], which means that each of block_size numbers in batch_size rows now represented
        # as an array of 64 random numbers. but why we need 64 numbers? Because we have so many unique characters in dataset
        # and each number indicates a probability for that number (but for now probabilities are random)
        logits = self.token_embedding_table(idx)
        # now when we have predictions about what comes next, we would like to measure a loss (quality of predictions)
        # using cross_entropy (negative log likelihood), in order to do that we need to reshape logits and targets
        # (only because cross_entropy function expects data in different shapes)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_idx):
        # idx is [B, T] tensor of indices
        for _ in range(max_new_idx):
            # get the predictions
            logits, loss = self(idx) # forward pass
            # focus only on the last time step
            logits = logits[:, -1, :] # logits now has shape of [B, C]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # [B, C]
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # [B, 1]
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # [B, T+1]
        return idx

model = BigramLanguageModel(vocab_size) # initializa the model
model = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# training loop
for step in range(steps):
    # the loss inside training loop is a very noisy measurement of the current loss,
    # because every batch will be more or less lucky, so we will use estimate_loss function
    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    # sample a batch of data
    x, y = get_batch("train")
    # evaluate the loss
    logits, loss = model(x, y) # forward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate a sample
context = torch.zeros((1, 1), dtype=torch.long, device=device) # [[0]] - it is a reasonable starting context because 0 maps to a new line character
print(decode(model.generate(context, max_new_idx=500)[0].tolist()))

-- dataset has 65 unique characters: {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
step 0: train loss 4.7305, val loss 4.7241
step 300: train loss 4.3818, val loss 4.3896
step 600: train loss 4.0801, val loss 4.0784
step 900: train loss 3.8066, val loss 3.8117
step 1200: train loss 3.5844, val loss 3.5850
step 1500: train loss 3.3757, val loss 3.3829
step 1800: train loss 3.2182, val loss 3.2218
step 2100: train loss 3.0817, val loss 3.0810
step 2400: train loss 2.